In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('uran.xls', header = 1)
df.tail()

In [ ]:
br = pd.read_excel('нефть-brent.xlsx', header = 0)
br.head()

In [ ]:
br.index = pd.to_datetime(br['Дата'],format='%m/%d/%y %I:%M%p')
brent_barrel = br.groupby(by=[ br.index.year,br.index.month]).mean().values[:-1]

In [ ]:
dollar = [76.3 , 77.23, 70.51, 66.69, 65.68, 65.31, 64.34, 64.92, 64.60, 62.68, 64.36,
 62.2,  59.95, 58.4,  58.1,  56.43, 57.17, 57.84, 59.67, 59.64, 57.69, 
 57.73, 58.92, 58.58, 56.78, 56.81, 57.03, 60.46, 62.2,  62.7,  62.88, 66.12,
 67.65, 65.88, 66.24, 67.3,  67.34, 65.86, 65.14, 64.61, 64.84, 64.23, 63.19,
 65.53, 64.98, 64.35, 63.86, 62.94,  61.78, 63.88, 73.31, 75.23, 72.68, 69.22,
 71.28, 73.8,  75.66, 77.59, 77,    74.05, 74.22, 74.38, 76.09, 74.04, 73.5, 73.98]

In [ ]:
df['dollar'] = dollar
df['oil'] = brent_barrel

In [ ]:
df[['sh1']] = df[['Spot среднее (долл./U3O8)']].shift(1)
df[['d1']] = df[['dollar']].shift(1)

In [ ]:
df[['Spot среднее (долл./U3O8)','sh1','d1', 'dollar','oil']].corr(method='spearman')

In [ ]:
X = df[['dollar','oil','sh1','d1']].values
y = df[['Spot среднее (долл./U3O8)']].values

In [ ]:
%matplotlib inline

In [ ]:
from catboost import CatBoostRegressor
from catboost import Pool, cv

model = CatBoostRegressor()
# Fit model
model.fit(X[:-1], y[:-1],
          eval_set=(X[[-1]], y[-1]),
          verbose=True,
          plot=True)

In [ ]:
model.predict(X[-1]), y[-1]

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.title(f'UranTimeLine')
plt.plot(df[['Trade Tech Exchange Value']], '-o', zorder=1)
plt.plot(df[['dollar']], '-o', zorder=1)

#mask = (df[['UxC U3O8 Spot']] > df[['UxC U3O8 Spot']].tail(12).mean())
#plt.scatter(df[mask].index, df[mask]['UxC U3O8 Spot'], label='аномальный участок', color='red',zorder=2)

plt.legend()
plt.xlabel('Month')
plt.ylabel('Price, $')
plt.show()

In [ ]:
import numpy as np
from scipy.stats import mstats
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(20, 10))
plt.title(f'UranTimeLine')
plt.plot(df[['Spot среднее (долл./U3O8)']], '-o', zorder=1, label= f'Spot среднее (долл./U3O8')    
plt.plot(df[['dollar']]/2, '-o', zorder=1, label= f'USD RUB')


# Calculate the quantiles column wise
quantiles = mstats.mquantiles(df[['Spot среднее (долл./U3O8)']].tail(12).values,
                              prob=[ 0.14, 0.86], axis=0)
# Plot it
#labels = ['15%', '85%']
#for i, q in enumerate(quantiles):  
#    plt.axhline(q,color='k', linestyle='--' , label= f'Перцентиль {labels[i]}'
#               )
plt.axhline(quantiles[0] ,color='k', linestyle='--' , label= f'Автоматически рассчитанный порог отклонения' )
plt.axhline(quantiles[1] ,color='k', linestyle='--'  )
    
last_value = df[['Spot среднее (долл./U3O8)']].tail(1).values[0]
year_mean = df[['Spot среднее (долл./U3O8)']].tail(12).values.mean()  


a = quantiles[0][0]
b = quantiles[1][0]
x = df[['Spot среднее (долл./U3O8)']].tail(1).values[0]
if not min(a, b) < x < max(a, b):
    plt.scatter(df[['Spot среднее (долл./U3O8)']].tail(1).index[0], 
                df[['Spot среднее (долл./U3O8)']].tail(1).values[0], 
                label='Аномальное значение', color='red',zorder=2)

# plt.scatter(df[['Spot среднее (долл./U3O8)']][47:59].idxmax()[0], 
#             max(df[['Spot среднее (долл./U3O8)']][47:59].values )[0], 
#             label='Максимальное значение, предыдущий год', color='red',zorder=2)   

# plt.scatter(df[['Spot среднее (долл./U3O8)']][47:59].idxmin()[0], 
#             min(df[['Spot среднее (долл./U3O8)']][47:59].values )[0], 
#             label='Минимальное значение, предыдущий год', color='red',zorder=2)  

# draw diagonal line from (70, 90) to (90, 200)
#plt.plot([48, 60],[max(df[['Spot среднее (долл./U3O8)']][47:59].values )[0], max(df[['Spot среднее (долл./U3O8)']][47:59].values )[0]], 'k-')
#plt.plot([48, 60],[min(df[['Spot среднее (долл./U3O8)']][47:59].values )[0], min(df[['Spot среднее (долл./U3O8)']][47:59].values )[0]], 'k-')




plt.plot([34, 52],[df[['Spot среднее (долл./U3O8)']][34:35].values[0][0], df[['Spot среднее (долл./U3O8)']][52:53].values[0][0] ], 'k-')
plt.plot([27, 48],[df[['Spot среднее (долл./U3O8)']][27:28].values[0][0], df[['Spot среднее (долл./U3O8)']][48:49].values[0][0] ], 'k-')




    
plt.axhline(year_mean * 1.1 ,color='r', linestyle='--', label= f'Максимальное отклонение, выбранное экспертом')
plt.axhline(year_mean * 0.9 ,color='r', linestyle='--')

plt.axvline(0 ,color='tab:blue', linestyle='--')
plt.axvline(12 ,color='tab:blue', linestyle='--')
plt.axvline(24 ,color='tab:blue', linestyle='--')
plt.axvline(36 ,color='tab:blue', linestyle='--')
plt.axvline(48 ,color='tab:blue', linestyle='--')
plt.axvline(60 ,color='tab:blue', linestyle='--')

plt.legend()



ax.set_ylabel('Spot среднее USD ')
ax.set_xlabel('Month \n ')

plt.show()

In [ ]:
print('Верхний порог ', round(year_mean * 1.1, 2)  )
print('Нижний порог', round(year_mean * 0.9, 2)  )

print('Максимальное отклонение от среднегодового значения', 
      round(max(df[['Spot среднее (долл./U3O8)']][47:59].values )[0] / year_mean -1,2)*100 , '%')